In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
demo_cleaned = pd.read_table(r'../data/demo_cleaned', sep = ',')
exp_cleaned = pd.read_table(r'../data/exp_cleaned', sep = ',')
web_cleaned = pd.read_table(r'../data/web_cleaned', sep = ',')

In [20]:
web_test.head(10)

,Unnamed: 0,client_id,visitor_id,visit_id,process_step,date_time,Variation
0,0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test
1,1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test
2,2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test
3,3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test
4,4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test
5,5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15,Test
6,6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01,Test
7,7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22,Test
8,8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05,Test
9,9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43,Test


In [15]:
test_join = web_cleaned.merge(exp_cleaned[['client_id', 'Variation']], on='client_id', how='left', suffixes=('', '_exp'), validate='many_to_one')

In [17]:
web_test = test_join[test_join['Variation'] == 'Test']
web_control = test_join[test_join['Variation'] == 'Control']

## Count backward steps and compare

In [35]:
web_test.head()

,Unnamed: 0,client_id,visitor_id,visit_id,process_step,date_time,Variation,backward_step
0,0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,False
1,1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,False
2,2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,True
3,3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,False
4,4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,True


In [31]:
# Create the shift dfs
web_test_shift = web_test.shift(1)
web_control_shift = web_control.shift(1)

backward_step = [('confirm','step_3'),('confirm','step_2'),('confirm','step_1'),('confirm','start'),
                 ('step_3','step_2'),('step_3','step_1'),('step_3','start'),
                 ('step_2','step_1'),('step_2','start'),
                 ('step_1','start')]

# For the test group
web_test.loc[:, 'backward_step'] = [step in backward_step for step in zip(web_test['process_step'], web_test_shift['process_step'])]
count_backward_steps_test = web_test['backward_step'].sum()

# For the control group
web_control.loc[:, 'backward_step'] = [step in backward_step for step in zip(web_control['process_step'], web_control_shift['process_step'])]
count_backward_steps_control = web_control['backward_step'].sum()

In [32]:
count_backward_steps_test

38675

In [33]:
count_backward_steps_control

29879

In [30]:
('step_3', 'step_2') in backward_step

True